In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

from tqdm.notebook import tqdm

import sys
sys.path.append('../')
import default_style

TS_DATASET_FOLDER = os.path.join("..", "dataset")
TS_PREPROC_FOLDER = os.path.join(TS_DATASET_FOLDER, "preprocessed_traces")
DF_PREPROC_FILE = os.path.join(TS_PREPROC_FOLDER, "preproc_ts.df")

INTERESTING_TRACES = ["denoised_trace", "std_phon_trace", "phonetic_trace"]

## Save folder for long-run estimators
ESTIMATORS_FOLDER = os.path.join("..", "estimators")

In [2]:
# Load stuff
df = pd.read_csv(DF_PREPROC_FILE)

traces = dict()
for t in INTERESTING_TRACES:
    traces[t] = np.load(os.path.join(TS_PREPROC_FOLDER, f"{t}.npy"))

In [7]:
# Minimal preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

X = traces["std_phon_trace"]
X[X==np.nan] = 0
X = X.reshape(-1, X.shape[1], 1) # Shape of LSTM is (num_timesteps, num_features)


ohenc = OneHotEncoder()
y = ohenc.fit_transform(df.emotion.values.reshape(-1,1)).toarray()


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [8]:
print(X_test.shape)
print(y_test.shape)

(491, 745, 1)
(491, 8)


In [9]:
# Define the model
from keras.layers import LSTM, GRU, Conv1D, Input, Dense, Flatten
from keras.metrics import CategoricalCrossentropy as cat_cross_metric, CategoricalAccuracy as cat_acc_metric
from keras.losses import CategoricalCrossentropy
from keras.models import Model

# Model
input_layer = Input(shape=(X.shape[1],1), name="input")
lstm = LSTM(128)(input_layer)
dense = Dense(128)(lstm)
flatten = Flatten()(lstm)
dense = Dense(128)(flatten)
output = Dense(y.shape[1], activation="softmax", name="output")(dense)

# Compilation
model = Model(inputs=input_layer, outputs=output, name="LSTM_dense")
model.compile(optimizer="adam",
            loss=CategoricalCrossentropy(from_logits=False),
            metrics= [cat_cross_metric(from_logits=False), cat_acc_metric()]
             )

model.summary()

Model: "LSTM_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 745, 1)]          0         
                                                                 
 lstm_1 (LSTM)               (None, 128)               66560     
                                                                 
 flatten_1 (Flatten)         (None, 128)               0         
                                                                 
 dense_3 (Dense)             (None, 128)               16512     
                                                                 
 output (Dense)              (None, 8)                 1032      
                                                                 
Total params: 84,104
Trainable params: 84,104
Non-trainable params: 0
_________________________________________________________________


2023-06-08 14:59:49.315988: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-06-08 14:59:49.318843: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-06-08 14:59:49.320838: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [11]:
# Fit
history = model.fit(
                    X_train, y_train, 
                    epochs=10, batch_size=10, verbose=1, 
                    validation_split = 0.05
                   )

Epoch 1/10


2023-06-08 14:59:56.249884: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-06-08 14:59:56.252753: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-06-08 14:59:56.255163: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

187/187 [==============================] - ETA: 0s - loss: 2.0427 - categorical_crossentropy: 2.0427 - categorical_accuracy: 0.1450

2023-06-08 15:01:43.192614: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-06-08 15:01:43.195529: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-06-08 15:01:43.198122: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

187/187 [==============================] - 109s 565ms/step - loss: 2.0427 - categorical_crossentropy: 2.0427 - categorical_accuracy: 0.1450 - val_loss: 2.0013 - val_categorical_crossentropy: 2.0013 - val_categorical_accuracy: 0.1717
Epoch 2/10
187/187 [==============================] - 103s 553ms/step - loss: 2.0317 - categorical_crossentropy: 2.0317 - categorical_accuracy: 0.1697 - val_loss: 2.0037 - val_categorical_crossentropy: 2.0037 - val_categorical_accuracy: 0.2323
Epoch 3/10
187/187 [==============================] - 107s 570ms/step - loss: 2.0223 - categorical_crossentropy: 2.0223 - categorical_accuracy: 0.1697 - val_loss: 1.9839 - val_categorical_crossentropy: 1.9839 - val_categorical_accuracy: 0.2222
Epoch 4/10
 25/187 [===>..........................] - ETA: 1:34 - loss: 2.0253 - categorical_crossentropy: 2.0253 - categorical_accuracy: 0.1840

KeyboardInterrupt: 